In [7]:
import json
import numpy as np
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE

# Load intents dataset
with open('/Users/macbook/Downloads/mental_health_intent_dataset.json') as file:
    data = json.load(file)

# Extract patterns and tags
patterns = []
tags = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        patterns.append(pattern)
        tags.append(intent['tag'])

# Encode target labels
label_encoder = LabelEncoder()
encoded_tags = label_encoder.fit_transform(tags)

# Text vectorization using TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(patterns).toarray()

# Balance the dataset using SMOTE
smote = SMOTE(random_state=42, k_neighbors=1)  # Set k_neighbors to 1 to handle small class sizes
X_balanced, y_balanced = smote.fit_resample(X, encoded_tags)


# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=42, stratify=y_balanced)

# Define an SVM model with Grid Search for hyperparameter tuning
svc = SVC(probability=True, random_state=42)

param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

grid_search = GridSearchCV(svc, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Best model from Grid Search
model = grid_search.best_estimator_

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

# Save the model, label encoder, and vectorizer
with open('chatbot_model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

with open('label_encoder.pkl', 'wb') as le_file:
    pickle.dump(label_encoder, le_file)

with open('vectorizer.pkl', 'wb') as vectorizer_file:
    pickle.dump(vectorizer, vectorizer_file)


Model Accuracy: 80.29%
Classification Report:
                                         precision    recall  f1-score   support

                         accepting_help       1.00      1.00      1.00         2
                  adapting_to_new_roles       1.00      1.00      1.00         2
                              addiction       1.00      1.00      1.00         2
                       anger_management       1.00      1.00      1.00         2
                        anxiety_general       1.00      1.00      1.00         2
                          assertiveness       1.00      1.00      1.00         2
                       boundary_setting       1.00      1.00      1.00         2
                        breaking_habits       1.00      1.00      1.00         2
                 building_better_habits       1.00      1.00      1.00         2
                       building_empathy       0.50      0.50      0.50         2
                    building_resilience       1.00      0.50  

/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Function to chat with the bot
def chatbot_response(user_input):
    input_vector = vectorizer.transform([user_input]).toarray()
    predicted_tag_index = model.predict(input_vector)[0]
    predicted_tag = label_encoder.inverse_transform([predicted_tag_index])[0]

    for intent in data['intents']:
        if intent['tag'] == predicted_tag:
            return np.random.choice(intent['responses'])

# Test the chatbot
print("\nChat with the bot (type 'quit' to stop):")
while True:
    user_input = input("You: ")
    if user_input.lower() in ["quit", "exit"]:
        break
    print(f"Bot: {chatbot_response(user_input)}")


Chat with the bot (type 'quit' to stop):


You:  hi


Bot: Would you like some tips on managing your daily routine? I’m here for you.


You:  I have headache


Bot: Taking a break and prioritizing self-care is essential. Shall we create a plan?


...

Bot: It’s okay to feel this way. Let’s take small steps together.


You:  ok


Bot: It’s okay to feel this way. Let’s take small steps together.


You:  I feel anxious about everything


Bot: Anxiety can be overwhelming. Talking to a therapist might help.
